In [26]:
## >> Tut : http://deeplearning.net/tutorial/logreg.html <<<

import cPickle
import gzip
import os
import sys
import timeit

import numpy as np

import theano
import theano.tensor as T


In [63]:
## Load Data ##
def load_data(dataset):
    data_dir, data_file = os.path.split(dataset)
    if data_dir == "" and not os.path.isfile(dataset):
        new_path = os.path.join(
            os.path.split(__file__)[0],
            "..",
            "data",
            dataset
        )
        if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
            dataset = new_path

    if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
        import urllib
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print 'Downloading data from %s' % origin
        urllib.urlretrieve(origin, dataset)

    print '... loading data'

    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    def shared_dataset(data_xy, borrow=True):
        data_x, data_y = data_xy
        shared_x = theano.shared(numpy.asarray(data_x,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        shared_y = theano.shared(numpy.asarray(data_y,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        return shared_x, T.cast(shared_y, 'int32')

    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval

In [49]:
# Logistic Regression Class

class LogisticReg(object):
    def __init__(self, input, n_in, n_out):
        self.w = theano.shared(
            value=numpy.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='w',
            borrow=True
        )
         # initialize the biases b as a vector of n_out 0s
        self.b = theano.shared(
            value=numpy.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )
        self.p_y_given_x = T.nnet.softmax(T.dot(input,self.w) + self.b)
        self.y_pred = np.argmax(self.p_y_given_x,axis = 1)
        self.params = [self.w, self.b]
        self.input = input

        
    def neg_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    
    def errors(self,y):
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()

In [57]:
## Instantiate the class

x = T.matrix('x')
y = T.ivector('y')

index = T.iscalar('index')

classifier = LogisticReg(input=x, n_in=28 * 28, n_out=10)

cost = classifier.neg_log_likelihood(y)

#classifier = Logistic(input=x,n_in=28*28,n_out=10)

In [64]:
#### Train symbolic function : Compile ####

# gradients
gw,gb = T.grad(cost,[classifier.w,classifier.b])

# Update weight and bias
updates=[(classifier.w, classifier.w - (0.01*gw) ),(classifier.b, classifier.b - (0.01*gb))]

# train function
train = theano.function(inputs=[x,y],
                        outputs=[cost],
                        updates = updates
                       )

In [70]:
## Load dataset ##
datasets = load_data('mnist.pkl.gz')

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x, test_set_y = datasets[2]

... loading data


In [83]:
print train_set_x.get_value().shape
print test_set_x.get_value().shape

#dir(train_set_y)
#print train_set_y.get_value().shape
#print valid_set_x.get_value().shape,valid_set_y.get_value().shape
#print test_set_x.get_value().shape,test_set_y.get_value().shape

(50000, 784)
(10000, 784)
